In [1]:
%defaultDatasource jdbc:h2:mem:db
DROP Table IF EXISTS ParticipacaoComite;
DROP Table IF EXISTS Atleta;
DROP Table IF EXISTS EsporteModalidade;
DROP TABLE IF EXISTS EdicaoDosJogos;
DROP TABLE IF EXISTS ComiteOlimpico;
DROP TABLE IF EXISTS ParticipacaoAtletas;
DROP TABLE IF EXISTS ComiteDosAtletas;
DROP TABLE IF EXISTS EsportesDasEdicoes;

In [2]:
CREATE TABLE EdicaoDosJogos (
    Ano INTEGER NOT NULL,
    NumeroDaEdicao INTEGER NOT NULL,
    CidadeSede VARCHAR(20) NOT NULL,
    TotalDeAtletas INTEGER NOT NULL,
    Mascote VARCHAR(50) NOT NULL,
    PRIMARY KEY (Ano)
) AS SELECT
    Ano,
    NumeroDaEdicao,
    Cidade,
    TotalDeAtletas,
    Mascote
FROM CSVREAD('./edicoes.csv');

In [3]:
CREATE TABLE Atleta (
    Id INTEGER NOT NULL,
    Nome VARCHAR(100) NOT NULL,
    AnoDeNascimento VARCHAR(10),
    Sexo VARCHAR(1),
    PRIMARY KEY (Id)
) AS SELECT
    Id,
    Nome,
    AnoDeNascimento,
    Sexo
FROM CSVREAD('./atletas.csv');

In [4]:
CREATE TABLE ComiteOlimpico (
    Sigla VARCHAR(3) NOT NULL,
    Pais VARCHAR(50) NOT NULL,
    PRIMARY KEY (Sigla)
) AS SELECT
    Id,
    País
FROM CSVREAD('./comites.csv');

In [5]:
CREATE TABLE EsporteModalidade (
    Id INTEGER NOT NULL,
    Nome VARCHAR(100) NOT NULL,
    EsportePai VARCHAR(100),
    PRIMARY KEY (Id)
) AS SELECT
    Id,
    Nome,
    EsportePai
FROM CSVREAD('./esportes.csv');

In [6]:
CREATE TABLE ParticipacaoComites  (
    IdComite VARCHAR(3) NOT NULL,
    AnoEdicao INTEGER NOT NULL,
    QtdAtletas INTEGER NOT NULL,
    QtdOuro INTEGER NOT NULL,
    QtdPrata INTEGER NOT NULL,
    QtdBronze INTEGER NOT NULL,
    Classificacao VARCHAR(3) NOT NULL,
) AS SELECT
    IdComite,
    AnoEdicao,
    QtdAtletas,
    QtdOuro,
    QtdPrata,
    QtdBronze,
    Classificacao
FROM CSVREAD('./participacaoComites.csv')

In [7]:
CREATE TABLE ParticipacaoAtletas  (
    IdAtleta INTEGER NOT NULL,
    AnoEdicao INTEGER NOT NULL,
    IdModalidade INTEGER NOT NULL,
    Altura VARCHAR(16) NOT NULL,
    Peso VARCHAR(16) NOT NULL,
    Medalha VARCHAR(6) NOT NULL,
) AS SELECT
    IdAtleta,
    AnoEdicao,
    IdModalidade,
    Altura,
    Peso,
    Medalha
FROM CSVREAD('./participacaoAtletas.csv')

In [8]:
CREATE TABLE ComiteDosAtletas  (
    IdComite VARCHAR(3) NOT NULL,
    IdAtleta INTEGER NOT NULL,
) AS SELECT
    IdComite,
    IdAtleta
FROM CSVREAD('./comiteDosAtletas.csv')

In [9]:
CREATE TABLE EsportesDasEdicoes  (
    AnoEdicao INTEGER NOT NULL,
    IdModalidade INTEGER NOT NULL,
    Ouro VARCHAR(3),
    Prata VARCHAR(3),
    Bronze VARCHAR(3),
) AS SELECT
    AnoEdicao,
    IdModalidade,
    Ouro,
    Prata,
    Bronze
FROM CSVREAD('./esportesDasEdicoes.csv')

In [16]:
/*Pergunta 1*/

SELECT E.Ouro, COUNT(*) QtdOuro
    FROM EsportesDasEdicoes E, EsporteModalidade M
    WHERE M.Id=E.IdModalidade and M.Nome='Archery Women''s Individual'
    GROUP BY E.Ouro;

In [18]:
/*Pergunta 2*/

SELECT AnoEdicao, ROUND(AVG(CAST (Altura as float)),1) Media_altura
    FROM ParticipacaoAtletas
    WHERE AnoEdicao =2016 and Altura<>'-'
    GROUP BY AnoEdicao;

SELECT * 
    FROM (
        SELECT E.EsportePai, E.Nome, ROUND(AVG(CAST (Altura as float)),1) Media_Altura
        FROM ParticipacaoAtletas P, EsporteModalidade E
        WHERE P.IdModalidade=E.Id and AnoEdicao=2016 and Altura<>'-'
        GROUP BY IdModalidade
        )
    ORDER BY MEDIA_ALTURA DESC

In [19]:
/*Pergunta 2*/

SELECT AnoEdicao, ROUND(AVG(CAST (Peso as float)),1) Media_peso
    FROM ParticipacaoAtletas
    WHERE AnoEdicao =2016 and Peso<>'-'
    GROUP BY AnoEdicao;

SELECT * 
    FROM (
        SELECT E.EsportePai, E.Nome, ROUND(AVG(CAST (Peso as float)),1) Media_peso
        FROM ParticipacaoAtletas P, EsporteModalidade E
        WHERE P.IdModalidade=E.Id and AnoEdicao=2016 and Peso<>'-'
        GROUP BY IdModalidade
        )
    ORDER BY MEDIA_peso DESC

In [22]:
/*Pergunta 3*/
DROP TABLE IF EXISTS IDADE;
CREATE VIEW Idade AS
SELECT P.IdAtleta, P.AnoEdicao, (P.AnoEdicao-cast (A.AnoDeNascimento as float)) AS idade
    FROM Atleta A, ParticipacaoAtletas P
    WHERE A.Id=P.IdAtleta and A.AnoDeNascimento<>'-';

SELECT AnoEdicao, ROUND(AVG(idade),0) Media_Idade
    FROM Idade
    GROUP BY AnoEdicao
    ORDER BY AnoEdicao

In [24]:
/*Pergunta 4*/
SELECT AnoEdicao, IdComite, Classificacao
    FROM ParticipacaoComites
    WHere (IdComite='USA' OR IdComite='URS') AND AnoEdicao>1947 AND AnoEdicao< 1989;

In [26]:
/*Pergunta 5*/
SELECT P.ANOEDICAO, A.SEXO, COUNT(*) TOTAL
    FROM ATLETA A, PARTICIPACAOATLETAS P
    WHERE A.ID=P.IDATLETA
    GROUP BY P.ANOEDICAO, A.SEXO
    ORDER BY P.ANOEDICAO

In [11]:
/*Pergunta Extra 1*/
SELECT DISTINCT C.PAIS, CAST (P.Classificacao AS INT) Classificacao
    FROM ParticipacaoComites P, ComiteOlimpico C
    WHERE C.Sigla=P.IdComite AND AnoEdicao=2016 AND Classificacao<>'-'
    ORDER BY Classificacao

In [12]:
/*Pergunta Extra 2*/
SELECT C.PAIS, (SUM(P.QTDOURO)+SUM(P.QTDPRATA)+SUM(P.QTDBRONZE))/COUNT(*) MEDIA
    FROM PARTICIPACAOCOMITES P, COMITEOLIMPICO C
    WHERE P.IDCOMITE=C.SIGLA
    GROUP BY C.PAIS
    ORDER BY MEDIA DESC

In [36]:
/*Pergunta Extra 3*/
SELECT A.NOME, P.MEDALHA, COUNT(*) TOTAL
    FROM PARTICIPACAOATLETAS P, ATLETA A
    WHERE P.IDATLETA=A.ID AND A.ID=1467 AND P.MEDALHA<>'-'
    GROUP BY P.IDATLETA, P.MEDALHA

In [21]:
/*Pergunta Extra 3*/
SELECT NOME, COUNT(*) TOTALOLIMPIADAS
    FROM (SELECT A.NOME, COUNT(*) TOTALJOGOS
            FROM PARTICIPACAOATLETAS P, ATLETA A
            WHERE P.IDATLETA=A.ID AND A.ID=1467
            GROUP BY P.IDATLETA, P.ANOEDICAO)
    GROUP BY NOME

In [23]:
/*Pergunta extra 5*/

SELECT IdComite,AnoEdicao,Classificacao
    FROM ParticipacaoComites
    WHERE IdComite='RUS'
    ORDER BY AnoEdicao;

In [25]:
/*Pergunta extra 6*/

SELECT * 
    FROM ParticipacaoComites
    WHERE AnoEdicao = 1936 AND IdComite='GER';

SELECT E2.ESPORTEPAI, E2.NOME
    FROM EsportesDasEdicoes E1, EsporteModalidade E2 
    WHERE (Ouro='GER' OR Prata='GER' OR Bronze='GER') AND AnoEdicao=1936 and E1.IDMODALIDADE=E2.ID